In [21]:
import pandas as pd

# Cargar el dataset completo
ruta_archivo = "C:/Users/jrgsi/OneDrive/Escritorio/PROUECTOS HENRY/SISTEMA DE RECOMENDACION PELICULAS/datos/data_peliculas.csv"
df = pd.read_csv(ruta_archivo)

# Convertir la columna de fecha a tipo datetime para trabajar con años
df['fecha_estreno'] = pd.to_datetime(df['fecha_estreno'], errors='coerce')

# Filtrar por géneros populares
generos_populares = ['Drama', 'Comedy', 'Action', 'Adventure', 'Horror', 'Crime', 'Thriller', 'Animation', 'Fantasy', 'Romance']
df = df[df[generos_populares].any(axis=1)]  # Mantener solo películas con al menos uno de los géneros populares

# Ordenar por popularidad, promedio de votos y porcentaje de ingresos en orden descendente
df = df.sort_values(by=['popularidad', 'promedio_votos', 'porcentage_ganancias'], ascending=False)

# Limitar el conjunto a las primeras 50,000 películas de acuerdo a los criterios de relevancia
df_filtrado = df.head(50000)

# Guardar el DataFrame filtrado en un archivo CSV
df_filtrado.to_csv("C:/Users/jrgsi/OneDrive/Escritorio/PROUECTOS HENRY/SISTEMA DE RECOMENDACION PELICULAS/datos/data_peliculas_reducido.csv", index=False)
print(f"Filtrado realizado. {len(df_filtrado)} películas guardadas en 'data_peliculas_reducido.csv'")


Filtrado realizado. 4939 películas guardadas en 'data_peliculas_reducido.csv'


In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import joblib

# 1. Cargar el DataFrame de películas reducido
try:
    ruta_archivo = "C:/Users/jrgsi/OneDrive/Escritorio/PROUECTOS HENRY/SISTEMA DE RECOMENDACION PELICULAS/datos/data_peliculas_reducido.csv"
    df = pd.read_csv(ruta_archivo)
    print("Archivo reducido cargado correctamente.")
except Exception as e:
    print("Error al cargar el archivo:", e)
    exit()

# 2. Seleccionar columnas numéricas y de géneros
columnas_numericas_y_genero = [
    'popularidad', 'promedio_votos', 'presupuesto', 'ingresos', 'porcentage_ganancias',
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Fantasy', 
    'Horror', 'Mystery', 'Romance', 'Science Fiction', 'Thriller', 'War'
]

# Verificar que las columnas existan en el DataFrame
for col in columnas_numericas_y_genero:
    if col not in df.columns:
        print(f"Columna faltante: {col}")
        exit()

# 3. Crear la matriz de características
movie_features = df[columnas_numericas_y_genero].fillna(0)

# 4. Reducir la dimensionalidad con PCA para optimizar el rendimiento
n_componentes = min(30, movie_features.shape[1])  # Reducir a 30 componentes
pca = PCA(n_components=n_componentes)
movie_features_reducido = pca.fit_transform(movie_features)
print(f"Dimensiones reducidas a {n_componentes} componentes con PCA.")

# 5. Crear el modelo de vecinos más cercanos usando similitud de coseno
nbrs = NearestNeighbors(n_neighbors=6, metric='cosine').fit(movie_features_reducido)

# Guardar el modelo de recomendación y los títulos de las películas
joblib.dump(nbrs, 'modelo_recomendacion_pca.pkl')
joblib.dump(df['titulo'], 'titulos.pkl')
print("Modelo guardado como 'modelo_recomendacion_pca.pkl' y títulos guardados como 'titulos.pkl'.")

# 6. Definir la función de recomendación
def recomendacion(titulo_pelicula, num_recomendaciones=5):
    # Verificar si la película existe en el DataFrame
    if titulo_pelicula not in df['titulo'].values:
        return f"La película '{titulo_pelicula}' no se encuentra en el catálogo."

    # Obtener el índice de la película en el DataFrame
    idx_pelicula = df[df['titulo'] == titulo_pelicula].index[0]
    
    # Buscar los vecinos más cercanos
    _, indices = nbrs.kneighbors([movie_features_reducido[idx_pelicula]])
    recomendaciones = df['titulo'].iloc[indices[0][1:num_recomendaciones+1]].values
    
    return recomendaciones

# 7. Solicitar al usuario que ingrese el nombre de una película
titulo_pelicula = input("Ingresa el nombre de la película para obtener recomendaciones: ").strip()

# Verificar que el título no esté vacío antes de continuar
if not titulo_pelicula:
    print("No se ingresó un nombre de película válido. Intenta nuevamente.")
else:
    # 8. Obtener y mostrar las recomendaciones
    recomendaciones = recomendacion(titulo_pelicula)
    if isinstance(recomendaciones, str):
        print(recomendaciones)
    else:
        print("Películas recomendadas:")
        for i, pelicula in enumerate(recomendaciones, start=1):
            print(f"{i}. {pelicula}")



Archivo reducido cargado correctamente.
Dimensiones reducidas a 18 componentes con PCA.
Modelo guardado como 'modelo_recomendacion_pca.pkl' y títulos guardados como 'titulos.pkl'.
Películas recomendadas:
1. Three Kings
2. The Man from U.N.C.L.E.
3. Joy
4. Cats & Dogs 2 : The Revenge of Kitty Galore
5. The Thin Red Line
